In [1]:
import pandas as pd
import os

# Actual

In [2]:
data_frames=[]
for i in range(0,31):
    df=pd.read_excel(r'E:\Desktop\Datasets\actual_july\15 min Dispatch July 2020.xlsx',sheet_name=i) 

    temp=list(df.columns)

    date = temp[8].date()
    df.columns = df.iloc[0]
    df=df.drop(0)
    df=df.drop(columns=df.columns[-3:],axis=1)
    x=df.iloc[:, 0]

    t=0
    for i in x:
        t=t+1
        if i=='Total MW':

            break
    df=df.iloc[:t-1, :]
    df1 = pd.DataFrame()
    import datetime as dt
    t1 = dt.datetime.strptime('00:00:00', '%H:%M:%S')
    for i in range (1,len(df.columns),2):
        df1[str(t1.time())]=df.iloc[:, i]+df.iloc[:, i+1]
        t2 = dt.datetime.strptime('00:30:00', '%H:%M:%S')
        time_zero = dt.datetime.strptime('00:00:00', '%H:%M:%S')
        x=(t1 - time_zero + t2).time()
        t1 = dt.datetime.strptime(str(x), '%H:%M:%S')
        
    m=df.iloc[:t-1, 0]
    list(m)
    df1.insert(0, 'Power_Plant', m)
    df2=pd.melt(df1,id_vars=['Power_Plant'])
    df2['Date']=date
    data_frames.append(df2)


In [3]:
from functools import reduce
df_merged_1 = reduce(lambda  left,right: pd.merge(left,right,how='outer'), data_frames)
df_merged_1.shape

(54144, 4)

In [4]:
df_merged_1.head(10000)

,Power_Plant,variable,value,Date
0,Canyon,00:00:00,20.0,2020-07-01
1,WPS,00:00:00,0.0,2020-07-01
2,Olax,00:00:00,10.0,2020-07-01
3,Nlax,00:00:00,91.0,2020-07-01
4,Polp,00:00:00,40.0,2020-07-01
...,...,...,...,...
9995,CEB Sup Power,09:00:00,0.0,2020-07-08
9996,VPOWER Hamb.,09:00:00,24.0,2020-07-08
9997,VPOWER Palle.,09:00:00,0.0,2020-07-08
9998,VPOWER Galle,09:00:00,10.0,2020-07-08


In [5]:
df_merged_1=df_merged_1.rename(columns = {'Power_Plant':'Plant','variable':'Time','value':'Power'})

In [6]:
df_merged_1.head()

,Plant,Time,Power,Date
0,Canyon,00:00:00,20.0,2020-07-01
1,WPS,00:00:00,0.0,2020-07-01
2,Olax,00:00:00,10.0,2020-07-01
3,Nlax,00:00:00,91.0,2020-07-01
4,Polp,00:00:00,40.0,2020-07-01


# Forecast

In [7]:
#get all the files in the forecast folder..........
def fileList(path):
    listOfFiles = os.listdir(path)
    files=[]
    
    for file in listOfFiles:
        fullPath = os.path.join(path, file) #full path for the file
        if os.path.isdir(fullPath): #if there are sub folders.....
            files = files + fileList(fullPath) #recursively loop through the sub folder
        else:
            files.append(fullPath)
                
    return files

all_the_files=fileList(r'E:\Desktop\forcast_july')

In [8]:
##############
data_frames=[]
for file in all_the_files:
    df=pd.read_excel(file)
    df.columns = df.iloc[0] #make columns as 0th row items..
    columns=df.columns
    date=columns[1].date() 
    df=df.drop(0) #drop 0th row items
    df.columns = df.iloc[0] #make columns as 0th row items..
    df=df.drop(1)  #drop 1st row items
    plants_temp=df.iloc[:,0] #all the items in plant column(this includes unnessasary items)
    plants=[]
    #get only the correct plants...
    for plant in plants_temp:
        if plant=='Total Generation':
            break
        else:
            plants.append(plant)
    df = df.loc[:, df.columns.notnull()] #remove nan columns.............
    z=len(plants) #only get the relevant rows...............
    df=df.head(z) #only get the dataframe which contains the needed rows...
    df.insert(0, 'Power_Plant', plants)
    df=df.drop(columns=df.columns[-1:],axis=1) #drop the last column.......
    df=pd.melt(df,id_vars=['Power_Plant']) #reshape the df using melt function...
    df['Date']=date
    data_frames.append(df)

In [9]:
#merge all the dataframes of all the days...
from functools import reduce
df_merged_2 = reduce(lambda  left,right: pd.merge(left,right,how='outer'), data_frames)
df_merged_2.shape

(62352, 4)

In [10]:
df_merged_2.head(2000)

,Power_Plant,1,value,Date
0,WPS,00:00:00,0.0,2020-07-01
1,Cany,00:00:00,0.0,2020-07-01
2,Olax,00:00:00,10.0,2020-07-01
3,Nlax,00:00:00,60.636,2020-07-01
4,Polp,00:00:00,78.479,2020-07-01
...,...,...,...,...
1995,PCCP II,22:30:00,270.0,2020-07-01
1996,PCCP III,22:30:00,270.0,2020-07-01
1997,KPS,22:30:00,0.0,2020-07-01
1998,CEB_EMERGENCY,22:30:00,0.0,2020-07-01


In [11]:
df_merged_2=df_merged_2.rename(columns = {'Power_Plant':'Plant',1:'Time','value':'Power'}) #rename columns...

In [12]:
df_merged_2.head()

,Plant,Time,Power,Date
0,WPS,00:00:00,0.0,2020-07-01
1,Cany,00:00:00,0.0,2020-07-01
2,Olax,00:00:00,10.0,2020-07-01
3,Nlax,00:00:00,60.636,2020-07-01
4,Polp,00:00:00,78.479,2020-07-01


In [13]:
#FINAL MERGE OF ACTUAL AND FORECAST
dfs=[df_merged_1,df_merged_2]
df_merged_final = reduce(lambda  left,right: pd.merge(left,right,how='outer'), dfs)
df_merged_final.shape

(116496, 4)

In [14]:
df_merged_final.head(110000)

,Plant,Time,Power,Date
0,Canyon,00:00:00,20.0,2020-07-01
1,WPS,00:00:00,0.0,2020-07-01
2,Olax,00:00:00,10.0,2020-07-01
3,Nlax,00:00:00,91.0,2020-07-01
4,Polp,00:00:00,40.0,2020-07-01
...,...,...,...,...
109995,Barge,23:30:00,60.0,2020-07-06
109996,ACE (Matara),23:30:00,0.0,2020-07-06
109997,Ace (Embilipitiya),23:30:00,17.16,2020-07-06
109998,NORTHERN_Power,23:30:00,0.0,2020-07-06


#CALCULATE THE DIFFERENCE.....